# SnakeQL
# An SQL like Database built entirely within Python

In [2]:
#libraries imported
import json
import os
import threading
import datetime
from decimal import Decimal
import ply.lex as lex
import ply.yacc as yacc

In [3]:
class DatabaseError(Exception):
    pass

In [4]:
class TableExistsError(DatabaseError):
    pass

In [5]:
class ColumnNotFoundError(DatabaseError):
    pass

In [6]:
class DataTypeError(DatabaseError):
    pass

In [7]:
class TransactionError(DatabaseError):
    pass

In [8]:
class Transaction:
    def __init__(self, isolation_level='READ COMMITTED'):
        self.operations = []
        self.active = False
        self.isolation_level = isolation_level

In [ ]:
class Database:
    def __init__(self, db_path='db'):
        self.db_path = db_path
        if not os.path.exists(db_path):
            os.makedirs(db_path)
        self.tables = {}
        self.locks = {}
        self.transaction = Transaction()
        self.global_lock = threading.Lock()
        
    def begin_transaction(self, isolation_level='READ COMMITTED'):
        if self.transaction.active:
            raise TransactionError("Transaction already active")
        self.transaction.active = True
        self.transaction.operations = []
        self.transaction.isolation_level = isolation_level
        print(f"Transaction started with isolation level {isolation_level}")

    def commit(self):
        if not self.transaction.active:
            raise TransactionError("No active transaction")
        for op in self.transaction.operations:
            op()
        self.transaction.active = False
        self.transaction.operations = []
        print("Transaction committed")

    def rollback(self):
        if not self.transaction.active:
            raise TransactionError("No active transaction")
        self.transaction.active = False
        self.transaction.operations = []
        print("Transaction rolled back")

    def create_table(self, table_name, columns, index_columns=None):
        if table_name in self.tables:
            raise TableExistsError(f"Table '{table_name}' already exists")
        table = {
            'columns': columns,
            'rows': [],
            'indexes': {}
        }
        if index_columns:
            for col in index_columns:
                if col not in [c[0] for c in columns]:
                    raise ColumnNotFoundError(f"Index column '{col}' not in table columns")
                table['indexes'][col] = {}
        self.tables[table_name] = table
        self.locks[table_name] = threading.Lock()
        self._save_table(table_name)
        print(f"Table '{table_name}' created with columns {columns}")

    def drop_table(self, table_name):
        if table_name not in self.tables:
            raise TableNotFoundError(f"Table '{table_name}' does not exist")
        del self.tables[table_name]
        if os.path.exists(os.path.join(self.db_path, f"{table_name}.json")):
            os.remove(os.path.join(self.db_path, f"{table_name}.json"))
        print(f"Table '{table_name}' dropped.")

    def describe_table(self, table_name):
        table = self._get_table(table_name)
        print(f"Table '{table_name}' schema:")
        for col_name, col_type in table['columns']:
            print(f"- {col_name}: {col_type}")

    def alter_table_add(self, table_name, column_name, data_type):
        table = self._get_table(table_name)
        if column_name in [col[0] for col in table['columns']]:
            raise ColumnNotFoundError(f"Column '{column_name}' already exists in table '{table_name}'")
        table['columns'].append((column_name, data_type))
        for row in table['rows']:
            row[column_name] = None
        self._save_table(table_name)
        print(f"Table '{table_name}' altered. Column '{column_name}' added.")

    def alter_table_drop(self, table_name, column_name):
        table = self._get_table(table_name)
        if column_name not in [col[0] for col in table['columns']]:
            raise ColumnNotFoundError(f"Column '{column_name}' does not exist in table '{table_name}'")
        table['columns'] = [col for col in table['columns'] if col[0] != column_name]
        for row in table['rows']:
            if column_name in row:
                del row[column_name]
        self._save_table(table_name)
        print(f"Table '{table_name}' altered. Column '{column_name}' dropped.")

    def truncate_table(self, table_name):
        table = self._get_table(table_name)
        table['rows'] = []
        self._save_table(table_name)
        print(f"Table '{table_name}' truncated.")

    def insert_into(self, table_name, values):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            if len(values) != len(table['columns']):
                raise DatabaseError("Number of values does not match number of columns")
            row = {}
            for (col_name, col_type), value_str in zip(table['columns'], values):
                value = parse_value(value_str, col_type)
                row[col_name] = value
            self._update_indexes(table, row, add=True)
            if self.transaction.active:
                def op():
                    table['rows'].append(row)
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                table['rows'].append(row)
                self._save_table(table_name)
            print(f"Inserted into '{table_name}': {row}")

    def select(self, table_name, columns=None, where=None, order_by=None, limit=None):
        with self._get_lock(table_name, shared=True):
            table = self._get_table(table_name)
            if columns is None or '*' in columns:
                columns = [col_name for col_name, col_type in table['columns']]
            result = []
            rows = table['rows']
            if where and isinstance(where, tuple) and where[0] == 'index':
                index_col, value = where[1], where[2]
                if index_col in table['indexes']:
                    index = table['indexes'][index_col]
                    rows = index.get(value, [])
                else:
                    pass
                where = None
            if where:
                rows = filter(where, rows)
            if order_by:
                reverse = False
                if order_by.startswith('-'):
                    reverse = True
                    order_by = order_by[1:]
                rows = sorted(rows, key=lambda x: x.get(order_by), reverse=reverse)
            if limit:
                rows = list(rows)[:limit]
            for row in rows:
                selected_row = {col: row.get(col) for col in columns}
                result.append(selected_row)
            print(f"Selected from '{table_name}':")
            for row in result:
                print(row)
            return result

    def update(self, table_name, set_values, where=None):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            updated_rows = 0
            for row in table['rows']:
                if where is None or where(row):
                    self._update_indexes(table, row, add=False)
                    for col, val_str in set_values.items():
                        col_type = self._get_column_type(table, col)
                        val = parse_value(val_str, col_type)
                        row[col] = val
                    self._update_indexes(table, row, add=True)
                    updated_rows += 1
            if self.transaction.active:
                def op():
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                self._save_table(table_name)
            print(f"Updated {updated_rows} rows in '{table_name}'")

    def delete_from(self, table_name, where=None):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            new_rows = []
            deleted_rows = 0
            for row in table['rows']:
                if where is None or not where(row):
                    new_rows.append(row)
                else:
                    self._update_indexes(table, row, add=False)
                    deleted_rows += 1
            table['rows'] = new_rows
            if self.transaction.active:
                def op():
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                self._save_table(table_name)
            print(f"Deleted {deleted_rows} rows from '{table_name}'")

    def _save_table(self, table_name):
        table = self.tables[table_name]
        table_data = {
            'columns': table['columns'],
            'rows': table['rows'],
            'indexes': {col: dict(index) for col, index in table.get('indexes', {}).items()}
        }
        with open(os.path.join(self.db_path, f"{table_name}.json"), 'w') as f:
            json.dump(table_data, f, default=str)

    def _load_table(self, table_name):
        try:
            with open(os.path.join(self.db_path, f"{table_name}.json"), 'r') as f:
                table_data = json.load(f)
                table_data['rows'] = [
                    {col: parse_loaded_value(val, col_type)
                     for (col, col_type), val in zip(table_data['columns'], row.values())}
                    for row in table_data['rows']
                ]
                table_data['indexes'] = {
                    col: {parse_loaded_value(key, self._get_column_type(table_data, col)): val
                          for key, val in index.items()}
                    for col, index in table_data.get('indexes', {}).items()
                }
                self.tables[table_name] = table_data
        except FileNotFoundError:
            raise TableNotFoundError(f"Table '{table_name}' does not exist")

    def _get_table(self, table_name):
        if table_name not in self.tables:
            self._load_table(table_name)
        return self.tables[table_name]

    def _get_lock(self, table_name, shared=False):
        if table_name not in self.locks:
            self.locks[table_name] = threading.Lock()
        lock = self.locks[table_name]
        return lock

    def _get_column_type(self, table, column_name):
        for col_name, col_type in table['columns']:
            if col_name == column_name:
                return col_type
        raise ColumnNotFoundError(f"Column '{column_name}' does not exist")

    def _update_indexes(self, table, row, add=True):
        for index_col, index in table.get('indexes', {}).items():
            key = row.get(index_col)
            if key is None:
                continue
            if add:
                if key not in index:
                    index[key] = []
                index[key].append(row)
            else:
                if key in index and row in index[key]:
                    index[key].remove(row)
                    if not index[key]:
                        del index[key]


In [9]:
def parse_value(value_str, data_type):
    value_str = value_str.strip()
    if data_type == 'int':
        return int(value_str)
    elif data_type == 'float':
        return float(value_str)
    elif data_type == 'str':
        if value_str.startswith('"') and value_str.endswith('"'):
            return value_str[1:-1]
        else:
            raise DataTypeError(f"Invalid string value: {value_str}")
    elif data_type == 'bool':
        if value_str.lower() in ('true', 'false'):
            return value_str.lower() == 'true'
        else:
            raise DataTypeError(f"Invalid boolean value: {value_str}")
    elif data_type == 'date':
        try:
            return datetime.datetime.strptime(value_str.strip('"'), '%Y-%m-%d').date()
        except ValueError:
            raise DataTypeError(f"Invalid date format: {value_str}")
    elif data_type == 'datetime':
        try:
            return datetime.datetime.strptime(value_str.strip('"'), '%Y-%m-%d %H:%M:%S')
        except ValueError:
            raise DataTypeError(f"Invalid datetime format: {value_str}")
    else:
        raise DataTypeError(f"Unsupported data type: {data_type}")

In [10]:
def parse_loaded_value(value_str, data_type):
    if data_type == 'int':
        return int(value_str)
    elif data_type == 'float':
        return float(value_str)
    elif data_type == 'str':
        return value_str
    elif data_type == 'bool':
        return value_str == 'True'
    elif data_type == 'date':
        return datetime.datetime.strptime(value_str, '%Y-%m-%d').date()
    elif data_type == 'datetime':
        return datetime.datetime.strptime(value_str, '%Y-%m-%d %H:%M:%S')
    else:
        return value_str


In [12]:
tokens = (
    'SELECT', 'FROM', 'WHERE', 'AND', 'OR', 'NOT', 'INSERT', 'INTO', 'VALUES',
    'UPDATE', 'SET', 'DELETE', 'CREATE', 'TABLE', 'DROP', 'ALTER', 'ADD', 'COLUMN',
    'INDEXED', 'BY', 'DESCRIBE', 'TRUNCATE', 'ORDER', 'LIMIT',
    'JOIN', 'ON', 'AS', 'INT_TYPE', 'FLOAT_TYPE', 'STRING_TYPE', 'IDENTIFIER',
    'EQUALS', 'GREATER', 'LESS', 'GREATEREQ', 'LESSEQ', 'NOTEQ',
    'COMMA', 'LPAREN', 'RPAREN', 'SEMICOLON', 'DOT', 'NUMBER', 'QUOTED_STRING'
)

In [13]:
reserved = {
    'select': 'SELECT',
    'from': 'FROM',
    'where': 'WHERE',
    'and': 'AND',
    'or': 'OR',
    'not': 'NOT',
    'insert': 'INSERT',
    'into': 'INTO',
    'values': 'VALUES',
    'update': 'UPDATE',
    'set': 'SET',
    'delete': 'DELETE',
    'create': 'CREATE',
    'table': 'TABLE',
    'drop': 'DROP',
    'alter': 'ALTER',
    'add': 'ADD',
    'column': 'COLUMN',
    'indexed': 'INDEXED',
    'by': 'BY',
    'describe': 'DESCRIBE',
    'truncate': 'TRUNCATE',
    'order': 'ORDER',
    'limit': 'LIMIT',
    'join': 'JOIN',
    'on': 'ON',
    'as': 'AS',
    'int': 'INT_TYPE',
    'float': 'FLOAT_TYPE',
    'str': 'STRING_TYPE',
}


In [ ]:
tokens = list(tokens) + list(reserved.values())

t_EQUALS    = r'='
t_GREATER   = r'>'
t_LESS      = r'<'
t_GREATEREQ = r'>='
t_LESSEQ    = r'<='
t_NOTEQ     = r'!='
t_COMMA     = r','
t_LPAREN    = r'\('
t_RPAREN    = r'\)'
t_SEMICOLON = r';'
t_DOT       = r'\.'

t_ignore  = ' \t'


In [ ]:
def t_QUOTED_STRING(t):
    r'\"([^\\\n]|(\\.))*?\"'
    t.value = t.value[1:-1]
    return t

def t_NUMBER(t):
    r'\d+(\.\d+)?'
    if '.' in t.value:
        t.value = str(float(t.value))
    else:
        t.value = str(int(t.value))
    return t

def t_IDENTIFIER(t):
    r'[A-Za-z_][A-Za-z0-9_]*'
    t.type = reserved.get(t.value.lower(), 'IDENTIFIER')
    return t

def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)


In [ ]:
lexer = lex.lex()

In [ ]:
precedence = (
    ('left', 'OR'),
    ('left', 'AND'),
    ('right', 'NOT'),
    ('nonassoc', 'EQUALS', 'GREATER', 'LESS', 'GREATEREQ', 'LESSEQ', 'NOTEQ'),
)


In [11]:
def parse_condition(condition_str, table=None):
    condition_str = condition_str.strip()
    operators = ['>=', '<=', '!=', '=', '>', '<']
    for op in operators:
        if op in condition_str:
            col, val = condition_str.split(op, 1)
            col = col.strip()
            val = val.strip()
            if table:
                col_type = table._get_column_type(table._get_table(table_name), col)
                val = parse_value(val, col_type)
            else:
                val = val.strip('"')
            if op == '=':
                return lambda row: row.get(col) == val
            elif op == '>':
                return lambda row: row.get(col) > val
            elif op == '<':
                return lambda row: row.get(col) < val
            elif op == '>=':
                return lambda row: row.get(col) >= val
            elif op == '<=':
                return lambda row: row.get(col) <= val
            elif op == '!=':
                return lambda row: row.get(col) != val
    raise DatabaseError(f"Unsupported condition '{condition_str}'")



In [ ]:
def parse_command(command, db):
    tokens = command.strip().split()
    if not tokens:
        return None
    cmd = tokens[0].upper()
    try:
        if cmd == 'CREATE':
            if tokens[1].upper() != 'TABLE':
                raise DatabaseError("Syntax error: expected 'TABLE'")
            table_name = tokens[2]
            rest = ' '.join(tokens[3:]).strip()
            index_columns = None
            if 'INDEXED BY' in rest.upper():
                rest, index_part = rest.upper().split('INDEXED BY')
                index_columns = [col.strip() for col in index_part.strip().split(',')]
            if not rest.startswith('(') or not rest.endswith(')'):
                raise DatabaseError("Syntax error: expected columns in parentheses")
            columns_str = rest[1:-1]
            columns = []
            for col_def in columns_str.split(','):
                col_def = col_def.strip()
                if ':' not in col_def:
                    raise DatabaseError(f"Syntax error in column definition '{col_def}'")
                col_name, col_type = col_def.split(':', 1)
                columns.append((col_name.strip(), col_type.strip()))
            return ('CREATE_TABLE', table_name, columns, index_columns)
        elif cmd == 'DROP':
            if tokens[1].upper() != 'TABLE':
                raise DatabaseError("Syntax error: expected 'TABLE'")
            table_name = tokens[2]
            return ('DROP_TABLE', table_name)
        elif cmd == 'DESCRIBE':
            if tokens[1].upper() != 'TABLE':
                raise DatabaseError("Syntax error: expected 'TABLE'")
            table_name = tokens[2]
            return ('DESCRIBE_TABLE', table_name)
        elif cmd == 'ALTER':
            if tokens[1].upper() != 'TABLE':
                raise DatabaseError("Syntax error: expected 'TABLE'")
            table_name = tokens[2]
            if tokens[3].upper() == 'ADD':
                if tokens[4].upper() != 'COLUMN':
                    raise DatabaseError("Syntax error: expected 'COLUMN'")
                col_def = ' '.join(tokens[5:])
                if ':' not in col_def:
                    raise DatabaseError(f"Syntax error in column definition '{col_def}'")
                col_name, col_type = col_def.split(':', 1)
                return ('ALTER_TABLE_ADD', table_name, col_name.strip(), col_type.strip())
            elif tokens[3].upper() == 'DROP':
                if tokens[4].upper() != 'COLUMN':
                    raise DatabaseError("Syntax error: expected 'COLUMN'")
                col_name = tokens[5]
                return ('ALTER_TABLE_DROP', table_name, col_name.strip())
            else:
                raise DatabaseError("Syntax error: expected 'ADD' or 'DROP'")
        elif cmd == 'TRUNCATE':
            if tokens[1].upper() != 'TABLE':
                raise DatabaseError("Syntax error: expected 'TABLE'")
            table_name = tokens[2]
            return ('TRUNCATE_TABLE', table_name)
        elif cmd == 'INSERT':
            if tokens[1].upper() != 'INTO':
                raise DatabaseError("Syntax error: expected 'INTO'")
            table_name = tokens[2]
            if tokens[3].upper() != 'VALUES':
                raise DatabaseError("Syntax error: expected 'VALUES'")
            rest = ' '.join(tokens[4:]).strip()
            if not rest.startswith('(') or not rest.endswith(')'):
                raise DatabaseError("Syntax error: expected values in parentheses")
            values_str = rest[1:-1]
            values = []
            for val in values_str.split(','):
                val = val.strip()
                values.append(val)
            return ('INSERT_INTO', table_name, values)
        elif cmd == 'SELECT':
            select_index = tokens.index('SELECT')
            from_index = tokens.index('FROM')
            columns_str = ' '.join(tokens[select_index+1:from_index]).strip()
            columns = [col.strip() for col in columns_str.split(',')]
            table_name = tokens[from_index+1]
            where_func = None
            order_by = None
            limit = None
            if 'WHERE' in tokens:
                where_index = tokens.index('WHERE')
                condition_str = ' '.join(tokens[where_index+1:])
                if 'ORDER BY' in condition_str.upper():
                    condition_str, order_part = condition_str.upper().split('ORDER BY')
                    order_by = order_part.strip()
                if 'LIMIT' in condition_str.upper():
                    condition_str, limit_part = condition_str.upper().split('LIMIT')
                    limit = int(limit_part.strip())
                where_func = parse_condition(condition_str)
            return ('SELECT', table_name, columns, where_func, order_by, limit)
        elif cmd == 'UPDATE':
            table_name = tokens[1]
            if 'SET' not in tokens:
                raise DatabaseError("Syntax error: expected 'SET'")
            set_index = tokens.index('SET')
            set_str = ''
            where_func = None
            if 'WHERE' in tokens:
                where_index = tokens.index('WHERE')
                set_str = ' '.join(tokens[set_index+1:where_index])
                condition_str = ' '.join(tokens[where_index+1:])
                where_func = parse_condition(condition_str)
            else:
                set_str = ' '.join(tokens[set_index+1:])
            set_pairs = {}
            for pair in set_str.split(','):
                if '=' not in pair:
                    raise DatabaseError(f"Syntax error in SET clause: '{pair}'")
                col, val = pair.split('=', 1)
                col = col.strip()
                val = val.strip()
                set_pairs[col] = val
            return ('UPDATE', table_name, set_pairs, where_func)
        elif cmd == 'DELETE':
            if tokens[1].upper() != 'FROM':
                raise DatabaseError("Syntax error: expected 'FROM'")
            table_name = tokens[2]
            where_func = None
            if 'WHERE' in tokens:
                where_index = tokens.index('WHERE')
                condition_str = ' '.join(tokens[where_index+1:])
                where_func = parse_condition(condition_str)
            return ('DELETE', table_name, where_func)
        elif cmd == 'BEGIN':
            if 'TRANSACTION' in tokens:
                if 'ISOLATION LEVEL' in command.upper():
                    iso_index = command.upper().index('ISOLATION LEVEL')
                    iso_level = command[iso_index+15:].strip()
                else:
                    iso_level = 'READ COMMITTED'
                return ('BEGIN_TRANSACTION', iso_level)
            else:
                raise DatabaseError("Syntax error: expected 'TRANSACTION'")
        elif cmd == 'COMMIT':
            return ('COMMIT',)
        elif cmd == 'ROLLBACK':
            return ('ROLLBACK',)
        else:
            raise DatabaseError(f"Unknown command '{cmd}'")
    except ValueError as ve:
        raise DatabaseError(f"Syntax error: {ve}")
    except DatabaseError as de:
        raise de


In [ ]:
def p_statement(p):
    '''statement : create_table_statement
                 | insert_statement
                 | select_statement
                 | update_statement
                 | delete_statement
                 | begin_transaction_statement
                 | commit_statement
                 | rollback_statement
                 | drop_table_statement
                 | alter_table_statement
                 | describe_table_statement
                 | truncate_table_statement
                 | SEMICOLON'''
    p[0] = p[1]

def p_create_table_statement(p):
    '''create_table_statement : CREATE TABLE IDENTIFIER LPAREN column_definitions RPAREN indexed_by_clause'''
    p[0] = ('CREATE_TABLE', p[3], p[5], p[7])

def p_column_definitions(p):
    '''column_definitions : column_definitions COMMA column_definition
                          | column_definition'''
    if len(p) == 4:
        p[0] = p[1] + [p[3]]
    else:
        p[0] = [p[1]]

def p_column_definition(p):
    '''column_definition : IDENTIFIER data_type'''
    p[0] = (p[1], p[2])

def p_data_type(p):
    '''data_type : INT_TYPE
                 | FLOAT_TYPE
                 | STRING_TYPE'''
    p[0] = p[1]

def p_indexed_by_clause(p):
    '''indexed_by_clause : INDEXED BY index_columns
                         | empty'''
    if len(p) == 4:
        p[0] = p[3]
    else:
        p[0] = None

def p_index_columns(p):
    '''index_columns : index_columns COMMA IDENTIFIER
                     | IDENTIFIER'''
    if len(p) == 4:
        p[0] = p[1] + [p[3]]
    else:
        p[0] = [p[1]]

def p_insert_statement(p):
    '''insert_statement : INSERT INTO IDENTIFIER VALUES LPAREN value_list RPAREN'''
    p[0] = ('INSERT_INTO', p[3], p[6])

def p_value_list(p):
    '''value_list : value_list COMMA value
                  | value'''
    if len(p) == 4:
        p[0] = p[1] + [p[3]]
    else:
        p[0] = [p[1]]

def p_value(p):
    '''value : NUMBER
             | QUOTED_STRING'''
    p[0] = p[1]

def p_select_statement(p):
    '''select_statement : SELECT select_columns FROM IDENTIFIER where_clause order_by_clause limit_clause'''
    p[0] = ('SELECT', p[4], p[2], p[5], p[6], p[7])

def p_select_columns(p):
    '''select_columns : MULTIPLY
                      | column_list'''
    p[0] = p[1]

def p_column_list(p):
    '''column_list : column_list COMMA IDENTIFIER
                   | IDENTIFIER'''
    if len(p) == 4:
        p[0] = p[1] + [p[3]]
    else:
        p[0] = [p[1]]

def p_where_clause(p):
    '''where_clause : WHERE condition
                    | empty'''
    if len(p) == 3:
        p[0] = p[2]
    else:
        p[0] = None

def p_order_by_clause(p):
    '''order_by_clause : ORDER BY IDENTIFIER
                       | empty'''
    if len(p) == 4:
        p[0] = p[3]
    else:
        p[0] = None

def p_limit_clause(p):
    '''limit_clause : LIMIT NUMBER
                    | empty'''
    if len(p) == 3:
        p[0] = int(p[2])
    else:
        p[0] = None

def p_condition(p):
    '''condition : expression'''
    p[0] = p[1]

def p_expression(p):
    '''expression : expression AND expression
                  | expression OR expression
                  | NOT expression
                  | LPAREN expression RPAREN
                  | comparison'''
    if len(p) == 4:
        if p[2].upper() == 'AND':
            p[0] = ('AND', p[1], p[3])
        elif p[2].upper() == 'OR':
            p[0] = ('OR', p[1], p[3])
        else:
            p[0] = p[2]
    elif len(p) == 3:
        p[0] = ('NOT', p[2])
    elif len(p) == 2:
        p[0] = p[1]
    else:
        p[0] = p[2]

def p_comparison(p):
    '''comparison : IDENTIFIER comparison_operator value'''
    p[0] = ('COMPARE', p[1], p[2], p[3])

def p_comparison_operator(p):
    '''comparison_operator : EQUALS
                           | GREATER
                           | LESS
                           | GREATEREQ
                           | LESSEQ
                           | NOTEQ'''
    p[0] = p[1]

def p_update_statement(p):
    '''update_statement : UPDATE IDENTIFIER SET set_clause where_clause'''
    p[0] = ('UPDATE', p[2], p[4], p[5])

def p_set_clause(p):
    '''set_clause : set_clause COMMA assignment
                  | assignment'''
    if len(p) == 4:
        p[0] = {**p[1], **p[3]}
    else:
        p[0] = p[1]

def p_assignment(p):
    '''assignment : IDENTIFIER EQUALS value'''
    p[0] = {p[1]: p[3]}

def p_delete_statement(p):
    '''delete_statement : DELETE FROM IDENTIFIER where_clause'''
    p[0] = ('DELETE', p[3], p[4])

def p_drop_table_statement(p):
    '''drop_table_statement : DROP TABLE IDENTIFIER'''
    p[0] = ('DROP_TABLE', p[3])

def p_alter_table_statement(p):
    '''alter_table_statement : ALTER TABLE IDENTIFIER ADD COLUMN IDENTIFIER data_type
                             | ALTER TABLE IDENTIFIER DROP COLUMN IDENTIFIER'''
    if p[4].upper() == 'ADD':
        p[0] = ('ALTER_TABLE_ADD', p[3], p[6], p[7])
    elif p[4].upper() == 'DROP':
        p[0] = ('ALTER_TABLE_DROP', p[3], p[6])

def p_describe_table_statement(p):
    '''describe_table_statement : DESCRIBE TABLE IDENTIFIER'''
    p[0] = ('DESCRIBE_TABLE', p[3])

def p_truncate_table_statement(p):
    '''truncate_table_statement : TRUNCATE TABLE IDENTIFIER'''
    p[0] = ('TRUNCATE_TABLE', p[3])

def p_begin_transaction_statement(p):
    '''begin_transaction_statement : BEGIN TRANSACTION isolation_level_clause'''
    p[0] = ('BEGIN_TRANSACTION', p[3])

def p_isolation_level_clause(p):
    '''isolation_level_clause : ISOLATION LEVEL IDENTIFIER
                              | empty'''
    if len(p) == 4:
        p[0] = p[3]
    else:
        p[0] = 'READ COMMITTED'

def p_commit_statement(p):
    '''commit_statement : COMMIT'''
    p[0] = ('COMMIT',)

def p_rollback_statement(p):
    '''rollback_statement : ROLLBACK'''
    p[0] = ('ROLLBACK',)

def p_empty(p):
    'empty :'
    pass

def p_error(p):
    if p:
        print(f"Syntax error at '{p.value}'")
    else:
        print("Syntax error at EOF")


In [ ]:
#main database usage function
def main():
    db = Database()
    while True:
        try:
            command = input("db> ")
            if command.lower() in ('exit', 'quit'):
                break
            parsed = parse_command(command, db)
            if parsed is None:
                continue
            cmd_type = parsed[0]
            if cmd_type == 'CREATE_TABLE':
                _, table_name, columns, index_columns = parsed
                db.create_table(table_name, columns, index_columns)
            elif cmd_type == 'DROP_TABLE':
                _, table_name = parsed
                db.drop_table(table_name)
            elif cmd_type == 'DESCRIBE_TABLE':
                _, table_name = parsed
                db.describe_table(table_name)
            elif cmd_type == 'ALTER_TABLE_ADD':
                _, table_name, col_name, col_type = parsed
                db.alter_table_add(table_name, col_name, col_type)
            elif cmd_type == 'ALTER_TABLE_DROP':
                _, table_name, col_name = parsed
                db.alter_table_drop(table_name, col_name)
            elif cmd_type == 'TRUNCATE_TABLE':
                _, table_name = parsed
                db.truncate_table(table_name)
            elif cmd_type == 'INSERT_INTO':
                _, table_name, values = parsed
                db.insert_into(table_name, values)
            elif cmd_type == 'SELECT':
                _, table_name, columns, where_func, order_by, limit = parsed
                db.select(table_name, columns, where_func, order_by, limit)
            elif cmd_type == 'UPDATE':
                _, table_name, set_pairs, where_func = parsed
                db.update(table_name, set_pairs, where_func)
            elif cmd_type == 'DELETE':
                _, table_name, where_func = parsed
                db.delete_from(table_name, where_func)
            elif cmd_type == 'BEGIN_TRANSACTION':
                _, iso_level = parsed
                db.begin_transaction(iso_level)
            elif cmd_type == 'COMMIT':
                db.commit()
            elif cmd_type == 'ROLLBACK':
                db.rollback()
            else:
                print(f"Unknown command type '{cmd_type}'")
        except DatabaseError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

if __name__ == '__main__':
    main()